NAME- SAKSHI

PRN- 23070243039

# <div style="text-align: center; color: purple; font-size: 36px;">WEATHER FORECASTING SYSTEM</div>



## INSTALLING LIBRARIES

In [1]:
!pip install requests
!pip install tk


## FETCHING WEATHER DATA

In [1]:
import requests
import json

# OpenWeather API Key 
API_KEY = '04754d42a571403e51f00e9ae851b328'
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

# Function to get weather data from OpenWeather API
def get_weather(city_name):
    url = f"{BASE_URL}?q={city_name}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        temperature = data['main']['temp']
        humidity = data['main']['humidity']
        description = data['weather'][0]['description']
        pressure = data['main']['pressure']
        wind_speed = data['wind']['speed']
        return temperature, humidity, description, pressure, wind_speed
    else:
        return None


## WEATHER RECOMMENDATION LOGIC

In [3]:
# Mapping of weather conditions to recommendations
weather_recommendations = {
    "clear sky": "The weather is clear and sunny. It's a good day to be outside, wear sunscreen!",
    "few clouds": "The weather is mostly clear with a few clouds. It's a nice day to go outdoors!",
    "scattered clouds": "The sky is a little cloudy. It might rain later, so keep an umbrella handy!",
    "broken clouds": "There are broken clouds in the sky. It might rain soon, so be prepared.",
    "shower rain": "It looks like it's going to rain. Don't forget to carry an umbrella!",
    "rain": "It's raining! Make sure to stay dry and carry an umbrella.",
    "thunderstorm": "There is a thunderstorm. It's best to stay indoors and avoid going outside.",
    "snow": "It's snowing! Wear warm clothes and stay safe.",
    "mist": "The visibility is low due to mist. Drive carefully and avoid outdoor activities.",
}

# Function to generate recommendations based on weather description
def generate_recommendation(weather_condition, temperature, wind_speed, humidity):
    recommendation = weather_recommendations.get(weather_condition.lower(), "Weather is unpredictable today. Stay prepared for anything.")
    
    # Additional recommendations based on temperature and wind speed
    if temperature < 10:
        recommendation += "\nIt's quite cold outside. Make sure to wear warm clothing."
    elif temperature > 30:
        recommendation += "\nIt's quite hot outside. Stay hydrated and wear light clothing."
    
    if wind_speed > 15:
        recommendation += "\nThere are high winds today. Be careful when going outside and avoid outdoor activities."
    
    if humidity > 80:
        recommendation += "\nThe humidity is quite high today. You might want to wear light and breathable clothing."
    
    return recommendation


## FETCHING CITY NAMES FROM JSON FILES

In [4]:
# Fetch city names from a JSON file 
def fetch_city_names():
    city_names = []
    try:
        # Example: Fetch city data from a local file 
        with open("D:/SIG(MSc)/DSSA sem3/AI/current.city.list.json", "r", encoding="utf-8") as file:
            city_data = json.load(file)
            for city in city_data:
                city_names.append(city['name'])
    except Exception as e:
        print(f"Error loading city names: {e}")
    
    return city_names


## GUI SETUP

In [5]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

# Function to fetch weather and display it in the main window
def fetch_weather():
    city_name = city_listbox.get(city_listbox.curselection())  # Get the selected city name
    
    # Fetch the weather data
    weather_data = get_weather(city_name)
    
    if weather_data:
        temperature, humidity, description, pressure, wind_speed = weather_data
        recommendation = generate_recommendation(description, temperature, wind_speed, humidity)
        
        # Display the weather and recommendation in the main window
        result_text = (f"City: {city_name}\n"
                       f"Temperature: {temperature} °C\n"
                       f"Humidity: {humidity}%\n"
                       f"Weather: {description}\n"
                       f"Pressure: {pressure} hPa\n"
                       f"Wind Speed: {wind_speed} m/s")
        
        result_label.config(text=result_text)
        
        # Show the recommendation in a new window
        show_recommendation_window(recommendation)
    else:
        messagebox.showerror("Error", "City not found or invalid city name.")

# Function to create a new window to display the recommendation
def show_recommendation_window(recommendation):
    recommendation_window = tk.Toplevel(root)
    recommendation_window.title("Weather Recommendation")
    
    recommendation_label = tk.Label(recommendation_window, text=recommendation, justify="left", padx=10, pady=10)
    recommendation_label.pack()

# Create the GUI
root = tk.Tk()
root.title("Weather Fetching App")

# Create and place widgets
label = tk.Label(root, text="Select City:")
label.pack(pady=10)

# Fetch the city names from the JSON file
cities = fetch_city_names()

# Create a scrollable Listbox for city selection
listbox_frame = tk.Frame(root)
scrollbar = tk.Scrollbar(listbox_frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

city_listbox = tk.Listbox(listbox_frame, selectmode=tk.SINGLE, height=10)
for city in cities:
    city_listbox.insert(tk.END, city)
city_listbox.pack(side=tk.LEFT, fill=tk.BOTH)

listbox_frame.pack(pady=10)

fetch_button = tk.Button(root, text="Fetch Weather", command=fetch_weather)
fetch_button.pack(pady=10)

result_label = tk.Label(root, text="", justify="left")
result_label.pack(pady=10)

# Run the Tkinter event loop
root.mainloop()
